In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lightkurve as lk
from scipy.optimize import curve_fit

from astropy.convolution import Gaussian1DKernel
from astropy.convolution import convolve
sys.path.insert(0, "../")

from michael import janet

Hi there. Welcome to the good place.


In [2]:
uni = pd.read_csv("/Users/oliver hall/Research/unicorn/data/unicorn.csv", index_col=0)

Index 258 is a good tester. Clear rotational signal, ambiguous peak mode.

In [7]:
j = janet.boot(uni, index=259)

In [ ]:
j.run()

Already have data downloaded for Gaia ID 2227100384482274048.
If you want to check for new data, run `janet.update()`.
### Running Simple Astropy Lomb-Scargle on Sector 17 on star 2227100384482274048 ###
### Completed Simple Astropy Lomb-Scargle for Sector 17 on star 2227100384482274048 ###
### Saved results ###
### Running Simple Astropy Lomb-Scargle on Sector 18 on star 2227100384482274048 ###
### Completed Simple Astropy Lomb-Scargle for Sector 18 on star 2227100384482274048 ###
### Saved results ###
### Running Simple Astropy Lomb-Scargle on Sector 24 on star 2227100384482274048 ###
### Completed Simple Astropy Lomb-Scargle for Sector 24 on star 2227100384482274048 ###
### Saved results ###
### Running Simple Astropy Lomb-Scargle on Sector 25 on star 2227100384482274048 ###
### Completed Simple Astropy Lomb-Scargle for Sector 25 on star 2227100384482274048 ###
### Saved results ###
### Running Wavelet Estimation on star 2227100384482274048 ###
